# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# download necessary NLTK data
import nltk
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
import re
import pandas as pd
from nltk.tokenize import word_tokenize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
# load data from database
df = pd.read_sql_table('Message','sqlite:///FigureEight.db')
X = df.message.values
y = df[df.columns[4:]].values
categories = df.columns[4:].tolist()

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X[0]

'Weather update - a cold front from Cuba that could pass over Haiti'

In [5]:
y[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### 2. Write a tokenization function to process your text data

In [6]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", "", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
from sklearn.metrics import classification_report
# convert predictions to a dataframe
y_pred_df = pd.DataFrame(y_pred, columns=categories)

# convert y_test to a DataFrame and select relevant columns
y_test_df = pd.DataFrame(y_test, columns=categories)
y_test_sel = y_test_df.loc[:, y_pred_df.columns]

for category in categories:
    print("Category: {}".format(category))
    print(classification_report(y_test_sel[category], y_pred_df[category]))

Category: related
             precision    recall  f1-score   support

          0       0.36      0.00      0.01      1243
          1       0.76      1.00      0.86      3993

avg / total       0.67      0.76      0.66      5236

Category: request
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      4385
          1       0.00      0.00      0.00       851

avg / total       0.70      0.84      0.76      5236

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5211
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5236

Category: aid_related
             precision    recall  f1-score   support

          0       0.59      1.00      0.74      3070
          1       0.00      0.00      0.00      2166

avg / total       0.34      0.59      0.43      5236

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'clf__estimator__n_estimators': [10, 20, 30],
    'clf__estimator__max_depth': [None, 10, 20],
    'clf__estimator__min_samples_split': [2, 3, 4],
    'clf__estimator__criterion': ['gini', 'entropy']
}

In [12]:
grid_search = GridSearchCV(pipeline, param_grid=parameters, cv=3, verbose=2)

In [13]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min remaining:    0.0s


[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, total= 1.4min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, total= 2.6min
[CV] clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__criterion=gini, clf__estimator__max_depth=None, clf__estimator__min_samples_split=2, clf__estimator

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 216.7min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [10, 20, 30], 'clf__estimator__max_depth': [None, 10, 20], 'clf__estimator__min_samples_split': [2, 3, 4], 'clf__estimator__criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [14]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best score: {grid_search.best_score_}")

Best parameters: {'clf__estimator__criterion': 'gini', 'clf__estimator__max_depth': None, 'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 20}
Best score: 0.20425897631779985


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred = grid_search.predict(X_test)

In [16]:
# convert predictions to a dataframe
y_pred_df = pd.DataFrame(y_pred, columns=categories)

# convert y_test to a DataFrame and select relevant columns
y_test_df = pd.DataFrame(y_test, columns=categories)
y_test_sel = y_test_df.loc[:, y_pred_df.columns]
# Calculate the overall accuracy of the model
accuracy = (y_pred == y_test).mean().mean()
print(f"Overall accuracy: {accuracy:.3f}")

Overall accuracy: 0.927


In [17]:
for category in categories:
    print("Category: {}".format(category))
    print(classification_report(y_test_sel[category], y_pred_df[category]))

Category: related
             precision    recall  f1-score   support

          0       0.36      0.00      0.01      1243
          1       0.76      1.00      0.86      3993

avg / total       0.67      0.76      0.66      5236

Category: request
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      4385
          1       0.00      0.00      0.00       851

avg / total       0.70      0.84      0.76      5236

Category: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5211
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      5236

Category: aid_related
             precision    recall  f1-score   support

          0       0.59      1.00      0.74      3070
          1       0.00      0.00      0.00      2166

avg / total       0.34      0.59      0.43      5236

Category: medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [18]:
import pickle

# Save the model to a file
with open("model.pkl", "wb") as file:
    pickle.dump(grid_search, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.